In [1]:
import pandas as pd
import numpy as np
import glob
from functools import reduce
import ipywidgets 
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly
from PIL import Image

def clean_cols(df):
    for i, columns_old in enumerate(df.columns.levels):
        columns_new = np.where(columns_old.str.contains('Unnamed'), '', columns_old)
        df.rename(columns=dict(zip(columns_old, columns_new)), level=i, inplace=True)


dir = "C:\\Users\\Corey\\"
doc_dir = "C:\\Users\\Corey\\coreyrastello\\docs\\"

In [2]:
#read it into a dataframe, use the right number of headers and index cols

df = pd.read_csv(rf"{dir}coreyrastello\data\premier-league\AllSeasons\NonMatchData\epl_player_season_stats_goal_shot_creation_AllSeasons_Season.csv", header=[0,1], index_col=[0,1,2,3])
sch = pd.read_csv(rf"{dir}coreyrastello\data\premier-league\AllSeasons\NonMatchData\epl_schedule_AllSeasons_Season.csv", index_col='game_id')
clean_cols(df)


In [3]:
df.head()

nation    pos age  \
                                                                             
league             season team    player                                     
ENG-Premier League 1819   Arsenal Aaron Ramsey              WAL     MF  27   
                                  Ainsley Maitland-Niles    ENG     DF  20   
                                  Alex Iwobi                NGA  FW,MF  22   
                                  Alexandre Lacazette       FRA     FW  27   
                                  Bernd Leno                GER     GK  26   

                                                          born   90s SCA  \
                                                                     SCA   
league             season team    player                                   
ENG-Premier League 1819   Arsenal Aaron Ramsey            1990  14.8  48   
                                  Ainsley Maitland-Niles  1997  11.0  19   
                                  Alex Iwobi              1996  21.9  84   
                                  Alexandre Lacazette     1991  27.8  84   
                                  Bernd Leno              1992  31.5   0   

                                                               SCA Types  \
                                                         SCA90  PassLive   
league             season team    player                                   
ENG-Premier League 1819   Arsenal Aaron Ramsey            3.25        41   
                                  Ainsley Maitland-Niles  1.73        16   
                                  Alex Iwobi              3.84        77   
                                  Alexandre Lacazette     3.02        55   
                                  Bernd Leno              0.00         0   

                                                                       ...  \
                                                         PassDead  TO  ...   
league             season team    player                               ...   
ENG-Premier League 1819   Arsenal Aaron Ramsey                  0   3  ...   
                                  Ainsley Maitland-Niles        1   1  ...   
                                  Alex Iwobi                    1   3  ...   
                                  Alexandre Lacazette           0  16  ...   
                                  Bernd Leno                    0   0  ...   

                                                                 GCA        \
                                                         Fld Def GCA GCA90   
league             season team    player                                     
ENG-Premier League 1819   Arsenal Aaron Ramsey             0   1  10  0.68   
                                  Ainsley Maitland-Niles   0   0   2  0.18   
                                  Alex Iwobi               0   0  11  0.50   
                                  Alexandre Lacazette      5   2  15  0.54   
                                  Bernd Leno               0   0   0  0.00   

                                                         GCA Types           \
                                                          PassLive PassDead   
league             season team    player                                      
ENG-Premier League 1819   Arsenal Aaron Ramsey                   9        0   
                                  Ainsley Maitland-Niles         2        0   
                                  Alex Iwobi                    11        0   
                                  Alexandre Lacazette           10        0   
                                  Bernd Leno                     0        0   

                                                                        
                                                         TO Sh Fld Def  
league             season team    player                                
ENG-Premier League 1819   Arsenal Aaron Ramsey            1  0   0   0  
                                  Ainsl

In [4]:
df = df.reset_index()

In [5]:
df = df.loc[df['90s'] > 10]
df = df.loc[df['pos'].isin(['FW','MF','MF,FW','FW,MF'])]
df = df.loc[df['season'].isin([1920,2021,2122,2223,2324,2425])]
df.pos.unique()


array(['FW', 'MF', 'FW,MF', 'MF,FW'], dtype=object)

In [7]:
fig = px.scatter(
    x = df['SCA']['SCA90']
    ,y=df['GCA']['GCA90']
    ,color=df['team']
    ,text=df['player']
    ,hover_data=[df['player'],df['nation']]
    ,animation_frame = df['season']
    ,animation_group = df['player']
    ,height=720
    ,width=1280
    ,title = f"SCA & GCA - Attacking Efficiency (90s > 10)"
    ,labels={
        "x":"<b>SCA90</b>",
        "y":"<b>GCA90</b>",
        "color":"Team",
        "hover_data_0":"Player",
        "hover_data_1":"Nation"
    },
    range_x=[-0.2,10],
    range_y=[-0.2,2]
    )

fig.update_layout(
    showlegend=True,
    images=[dict(
        source=Image.open(rf"{dir}coreyrastello\data\premier-league\logos\EPL.png"),
        xref="paper", yref="paper",
        x=1, y=1.03,
        sizex=0.2, sizey=0.2,
        xanchor="right", yanchor="bottom"
      )],
    margin=dict(l=20, r=20, t=120, b=20),
    paper_bgcolor='lavender',
    title={
        'font':{
            'family':'Arial',
            'size' :25,
            'color':'#1C2C5b'
        }}
)

fig.update_traces(textposition='bottom center')

fig.update_yaxes(showgrid=True, tickfont_family='Arial Black', tickfont_size=15, titlefont_size=20, titlefont_color='#1C2C5b', tickfont_color='#1C2C5b')
fig.update_xaxes(showgrid=True, tickfont_family='Arial Black', tickfont_size=15, titlefont_size=20, titlefont_color='#1C2C5b', tickfont_color='#1C2C5b')

fig.write_html(rf"{dir}coreyrastello\visualization\python\premierleague_SCA_GCA_animated.html")
fig.write_html(rf"{doc_dir}premierleague_SCA_GCA_animated.html")
fig.show()